<a href="https://colab.research.google.com/github/m109103/bert/blob/main/MyBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=78e58a1616455e38e55d46109aed31988e3d04398076b567ea789c91fb02c74c
  Stored in directory: /root/.cache/pip/wheels/a4/e8/45/842b3a39831261aef9154b907eacbc4ac99499a99ae829b06f
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12305 sha256=9112088e482783158283ca1486c3d35491eb8f4bf8fd3252be7bb27c216f4209
  Stored in directory: /root/.cache/pip/wheels/46/68/26/692ed21edd832833c3b0a0e21615bcacd99ca458b3f9ed571f
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3960 sha256=bb375ba2882162682877b6b1c21b7c33f2334782dc76e6ba15becd70bd7a6edf
  Stored in directory: /root/.cache/pip/wheels/81/67/b5/d847588d075895281e1cf5590f819bd4cf076a554872268bd5
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4668 sha256=47d42f6bd3b97f5cec33e496ac6af0e665bbee128b08f9a0c36c8a06

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.900214824000045
GPU (s):
0.041853807999927994
GPU speedup over CPU: 69x


In [ ]:
!pip uninstall keras-nightly

In [ ]:
!pip uninstall -y tensorflow

In [ ]:
!pip install tensorflow-gpu==2.2.0

In [ ]:
!pip install keras==2.3.0

In [ ]:
!pip install keras-bert==0.82.0

In [5]:
!nvidia-smi

Sat Apr 16 07:15:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |   1838MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive  
drive.mount('/content/gdrive')  

In [ ]:
!sudo lsb_release -a

In [7]:
import sys
import random
import numpy as np

from tensorflow.keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from keras_bert import (
    load_vocabulary,
    load_trained_model_from_checkpoint,
    Tokenizer,
    get_checkpoint_paths,
)
from keras_bert.datasets import get_pretrained, PretrainedList

In [8]:
SPLIT_PORTION = 0.8  # 指定訓練資料與測試資料的比例
BATCH_SIZE = 2  # batch size建議不要設得太大，不然很有可能out of memory
EPOCHS = 5 # epoch 5次其實就很夠了，當然你可以嘗試再大一點，只是訓練要更久

In [9]:
model_path = get_pretrained(PretrainedList.chinese_base)
paths = get_checkpoint_paths(model_path)
bert_model = load_trained_model_from_checkpoint(
    paths.config, paths.checkpoint, training=False, seq_len=None
)
bert_model.summary(line_length=120)

381902848/381892918 [==============================] - 5s 0us/step
Model: "model_1"
________________________________________________________________________________________________________________________
 Layer (type)                          Output Shape               Param #       Connected to                            
 Input-Token (InputLayer)              [(None, None)]             0             []                                      
                                                                                                                        
 Input-Segment (InputLayer)            [(None, None)]             0             []                                      
                                                                                                                        
 Embedding-Token (TokenEmbedding)      [(None, None, 768),        16226304      ['Input-Token[0][0]']                   
                                        (21128, 768)]                